REFERENCES

In [ ]:
# COVID19Py is a Python wrapper for the ExpDev07/coronavirus-tracker-api REST API

# github.com/ExpDev07/coronavirus-tracker-api
# It retrieves data directly from @ExpDev07's backend but it can also be set up to use a different backend

# csbs - www.csbs.org/information-covid-19-coronavirus - U.S. County data that comes from the 
# Conference of State Bank Supervisors

# jhu - https://github.com/CSSEGISandData/COVID-19 - Worldwide Data repository operated by the 
# Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE)

# Other References:
# pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html
# pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html
# pythonspot.com/matplotlib-bar-chart/
# stackoverflow.com/questions/38152356/matplotlib-dollar-sign-with-thousands-comma-tick-labels
# stackoverflow.com/questions/42049147/convert-list-to-pandas-dataframe-column
# Scatter Plots on Maps - plotly.com/python/scatter-plots-on-maps/

IMPORT PACKAGES

In [ ]:
import COVID19Py
import pandas as pd
from pandas import DataFrame
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import dash
import dash_core_components as dcc
import dash_html_components as html

In [ ]:
covid19 = COVID19Py.COVID19() #create instance
#covid19

In [ ]:
covid19 = COVID19Py.COVID19(data_source="jhu") #ask for dataset
#covid19

In [ ]:
latest = covid19.getLatest() #latest confirmed/deaths/recovered
#latest

CREATE PANDAS DATAFRAME FROM LOCATION DATA

In [ ]:
#stackoverflow.com/questions/38231591/splitting-dictionary-list-inside-a-pandas-column-into-separate-columns
#stackoverflow.com/questions/12376863/adding-calculated-columns-to-a-dataframe-in-pandas
#pythonexamples.org/pandas-dataframe-delete-column/

locations = covid19.getLocations() #location information
#type(locations)
locationsdforg = pd.DataFrame(locations)
locationsdf = locationsdforg.drop(['country_code', 'last_updated'], axis=1)
newlocationsdf = pd.concat([locationsdf.drop(['latest'], axis=1), locationsdf['latest'].apply(pd.Series)], axis=1)
newlocationsdf = pd.concat([newlocationsdf.drop(['coordinates'], axis=1), newlocationsdf['coordinates'].apply(pd.Series)], axis=1)
deathstopop = (newlocationsdf.deaths/newlocationsdf.country_population)*100
newlocationsdf['deaths/population (%)'] = deathstopop
confirmedtopop = (newlocationsdf.confirmed/newlocationsdf.country_population)*100
newlocationsdf['confirmed/population (%)'] = confirmedtopop
deathstoconfirmed = (newlocationsdf.deaths/newlocationsdf.confirmed)*100
newlocationsdf['deaths/confirmed (%)'] = deathstoconfirmed
newlocationsdf

CREATE PLOTLY MAP IN DASH

In [ ]:
newlocationsdf['text'] = newlocationsdf['province'] + '<br>' + newlocationsdf['country'] + '<br>' + newlocationsdf['confirmed'].astype(str)
                                                                                                                  
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'country names',
        lon = newlocationsdf['longitude'],
        lat = newlocationsdf['latitude'],
        text = newlocationsdf['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = False,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Reds',
            cmin = newlocationsdf['confirmed'].min(),
            color = newlocationsdf['confirmed'],
            cmax = newlocationsdf['confirmed'].max(),
            colorbar_title="Total Number Confirmed Cases"
        )))

fig.update_layout(
        title = 'Countries Affected by SARS-COV-2',
        geo = dict(
            scope='world',
            projection_type='natural earth',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
#fig.show()

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])
app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter